In [70]:
import os
import time

# spark imports
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS

# data science imports
import math
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

os.environ["SPARK_HOME"] = "/usr/local/Cellar/apache-spark/1.5.1/"  
os.environ["PYSPARK_PYTHON"]="/usr/local/bin/python3"

%matplotlib inline

In [71]:
# spark config
spark = SparkSession \
    .builder \
    .appName("tv show recommendation") \
    .config("spark.driver.maxResultSize", "96g") \
    .config("spark.driver.memory", "96g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.master", "local[12]") \
    .getOrCreate()
# get spark context
sc = spark.sparkContext

In [72]:
movies = spark.read.load("tv-shows-oficial.csv", format="csv", header=True, inferSchema=True)
ratings = spark.read.load("ratings-oficial.csv", format="csv", header=True, inferSchema=True)

In [56]:
movies.show(5)

+-------+--------------+
|movieId|         title|
+-------+--------------+
|      1|         90210|
|      2|        .9-1-1|
|      3|13 Reasons Why|
|      4|      24 Hours|
|      5|       30 Rock|
+-------+--------------+
only showing top 5 rows



In [57]:
ratings.show(5)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|   1123|   3.0|
|     1|    691|   4.0|
|     1|   1082|   4.0|
|     1|    351|   4.0|
|     1|    235|   5.0|
+------+-------+------+
only showing top 5 rows



In [73]:
# load data
movie_rating = sc.textFile('ratings-oficial.csv')
# preprocess data -- only need ["userId", "movieId", "rating"]
header = movie_rating.take(1)[0]
# rating_data = movie_rating \
#     .filter(lambda line: line!=header) \
#     .map(lambda line: line.split(",")) \
#     .map(lambda tokens: (int(tokens[0]), int(tokens[1]), float(tokens[2]))) \
#     .cache()
# # check three rows
# rating_data.take(3)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 74.0 failed 1 times, most recent failure: Lost task 0.0 in stage 74.0 (TID 114, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 2.7 than that in driver 3.7, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:452)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:588)
	at org.apache.spark.api.python.PythonRunner$$anon$1